In [348]:
import numpy as np
import json
import pickle
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer

In [349]:
train_data = []

with open('../data/domain1_train.json', 'r') as file:
    for line in file:
        data = json.loads(line)
        train_data.append(data)

In [350]:
X = np.array([entry["text"] for entry in train_data])
y = np.array([entry["label"] for entry in train_data])

<ipython-input-350-07b0c7f8f9f5>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X = np.array([entry["text"] for entry in train_data])


In [351]:
#split training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [352]:
# BOW n-grams
#vectorizer = HashingVectorizer(n_features=10000)
vectorizer = CountVectorizer(ngram_range=(1,2))

X_train_text = [' '.join(map(str, sample)) for sample in X_train]
X_test_text = [' '.join(map(str, sample)) for sample in X_test]

X_train_transformed = vectorizer.fit_transform(X_train_text)
X_test_transformed = vectorizer.transform(X_test_text)

In [353]:
logistic_model = LogisticRegression()

### 插入部分

In [357]:
param_grid = {'C': [0.1, 1, 10]}
grid_search = GridSearchCV(logistic_model, param_grid, cv=5)

In [358]:
grid_search = GridSearchCV(logistic_model, param_grid, cv=5)

In [359]:
grid_search.fit(X_train_transformed, y_train)

/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

GridSearchCV(cv=5, estimator=LogisticRegression(),
             param_grid={'C': [0.1, 1, 10]})

In [360]:
best_model = grid_search.best_estimator_
best_model.fit(X_train_transformed, y_train)


/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(C=10)

In [361]:
y_pred = best_model.predict(X_test_transformed)
accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {accuracy}")
# kaggle结果76.8%

Test Accuracy: 0.9302564102564103


### 插入结束

In [354]:
logistic_model.fit(X_train_transformed, y_train)

LogisticRegression()

In [355]:
y_pred = logistic_model.predict(X_test_transformed)

# output acc
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.926923076923077


In [362]:
with open('../data/output_csv/logistic_output.csv', 'w') as output_file:
    output_file.write('id,class\n')  

    # read in dataset
    with open('../data/test_set.json', 'r') as file:
        for line in file:
            entry = json.loads(line)
            text = entry["text"]

            # word embedding
            X_test = vectorizer.transform([" ".join(map(str, text))])

            # use logistic model to predict
            prediction = best_model.predict(X_test)

            # output csv
            output_file.write(f"{entry['id']},{prediction[0]}\n")